<a href="https://colab.research.google.com/github/billbch/TP1/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pascal-voc-writer
#!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pascal_voc_writer import Writer
import xml.etree.ElementTree as ET

import random

import cv2
# making a function to load the image
def load_img(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image
from matplotlib import pyplot as plt
import matplotlib.patches as patches

import albumentations as A

import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!cp /content/gdrive/MyDrive/TP1/Dataset_Full_sinGrietas_v2.zip /content
!cp /content/gdrive/MyDrive/TP1/Dataset_Full_grietas_v2.zip /content

In [ ]:
!mkdir /content/images
#!unzip -q Dataset_Full_sinGrietas_v2.zip -d /content/images/all
!unzip -q Dataset_Full_grietas_v2.zip -d /content/images/all
!mkdir /content/images/new
#!unzip -q Dataset_Full_sinGrietas_v2.zip -d /content/images/new
!unzip -q Dataset_Full_grietas_v2.zip -d /content/images/new

In [ ]:
# folder path
dir_path = r'/content/images/all/'
new_path = r'/content/images/new/'

annots_exist = False
while annots_exist == False:
    # list to store files
    jpg_files = []
    xml_files = []

    # Iterate directory
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)) and ('xml' not in path):
            jpg_files.append(path)

    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)) and ('xml' in path):
            xml_files.append(path)

    names = [x.split('.')[0] for x in jpg_files]
    xml_names = [x.split('.')[0] for x in xml_files]


    count_miss = 0
    for i in names:
        if i not in xml_names:
            writer = Writer(f'{i}.jpg', 720, 960)
            # write to file
            writer.save(f'{dir_path}{i}.xml')
            writer.save(f'{new_path}{i}.xml')
            count_miss +=1
    if count_miss > 0:
        print('Missing xml:', count_miss)
    else:
        print('Missing xml:', count_miss)
        annots_exist = True


jpg_files.sort()
xml_files.sort()
#print(jpg_files[0:5])
#print(xml_files[0:5])
print('Amount of images:', len(jpg_files))
print('Amount of xml:', len(xml_files))

Missing xml: 42
Missing xml: 0
Amount of images: 618
Amount of xml: 618


In [ ]:
transformFlip = A.Compose(
    [A.HorizontalFlip(p=1)
    ],bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']),
)

transformRotate = A.Compose(
    [A.RandomContrast(limit = [0.3, 0.33],p = 1),
     A.Rotate(p=1, limit=[10,13]),
    ],bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']),
)

transformRotate2 = A.Compose(
    [A.HorizontalFlip(p=1),
     A.RandomContrast(limit = [0.3, 0.33],p = 1),
     A.Rotate(p=1, limit=[10,13]),
    ],bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']),
)

/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [ ]:
def Augmentation(im_files, anot_files, flip, rotate, rotate2):

    for img_id in range(len(im_files)):
        print(jpg_files[img_id])
        image = load_img(f"/content/images/all/{jpg_files[img_id]}")

        # parse xml file
        tree = ET.parse(f"/content/images/all/{xml_files[img_id]}")
        root = tree.getroot() # get root object

        height = int(root.find("size")[0].text)
        width = int(root.find("size")[1].text)
        channels = int(root.find("size")[2].text)

        bbox_class = []
        bboxes = []

        for member in root.findall('object'):
            class_name = member[0].text # class name

            # bbox coordinates
            xmin = int(member[4][0].text)
            ymin = int(member[4][1].text)
            xmax = int(member[4][2].text)
            ymax = int(member[4][3].text)
            # store data in list
            bbox_class.append(class_name)
            bboxes.append([xmin, ymin, xmax, ymax])

        category_name_to_id = {'Hueco': 1, 'HuecoGrave': 2, 'Grieta': 3}
        category_id_to_name = {1: 'Hueco', 2: 'HuecoGrave', 3: 'Grieta'}

        category_ids = [category_name_to_id[class_lbl] for class_lbl in bbox_class]

        transformedFlip = flip(image=image, bboxes=bboxes,
                               category_ids=category_ids) # give image, bboxes and class_labels to transform
        transformedRot = rotate(image=image, bboxes=bboxes,
                                 category_ids=category_ids) # give image, bboxes and class_labels to transform
        transformedRot2 = rotate2(image=image, bboxes=bboxes,
                                 category_ids=category_ids) # give image, bboxes and class_labels to transform

        transforms = [transformedFlip, transformedRot, transformedRot2]

        # create pascal voc writer (image_path, width, height)
        new_id = 2
        name = jpg_files[img_id].split('.')[0]
        for t in transforms:
            writer = Writer(f'/my-project-name/{name}_{new_id}.jpg', image.shape[1], image.shape[0]) # y, x

            i_box = 0
            for box in transforms[new_id-2]['bboxes']:
                # add objects (class, xmin, ymin, xmax, ymax)
                writer.addObject(bbox_class[i_box], int(box[0]), int(box[1]), int(box[2]), int(box[3]))
                i_box += 1

            # write to file
            writer.save(f'/content/images/new/{name}_{new_id}.xml')
            cv2.imwrite(f'/content/images/new/{name}_{new_id}.jpg', cv2.cvtColor(transforms[new_id-2]['image'], cv2.COLOR_RGB2BGR))
            new_id +=1

In [ ]:
Augmentation(jpg_files, xml_files, transformFlip, transformRotate, transformRotate2)

IMG20230430213048.jpg
IMG20230430213057.jpg
IMG20230502205737.jpg
IMG20230504184323.jpg
IMG20230504184331.jpg
IMG20230504184416.jpg
IMG20230504184417.jpg
IMG20230504184426.jpg
IMG20230504184459.jpg
IMG20230504184612.jpg
IMG20230504184619.jpg
IMG20230504184636.jpg
IMG20230504184903.jpg
IMG20230504184909.jpg
IMG20230504210940.jpg
IMG20230504210952.jpg
IMG20230504210955.jpg
IMG20230504210958.jpg
IMG20230504211012.jpg
IMG20230504211019.jpg
IMG20230504211049.jpg
IMG20230504211104.jpg
IMG20230504211107.jpg
IMG20230504211119.jpg
IMG20230504211227.jpg
IMG20230504211232.jpg
IMG20230504211234.jpg
IMG20230504211241.jpg
IMG20230504211333.jpg
IMG20230507123206.jpg
IMG20230507123209.jpg
IMG20230507123211.jpg
IMG20230507123214.jpg
IMG20230507123224.jpg
IMG20230507123228.jpg
IMG20230507123303.jpg
IMG20230507123311.jpg
IMG20230507123316.jpg
IMG20230507123323.jpg
IMG20230507123459.jpg
IMG20230507123512.jpg
IMG20230507123515.jpg
IMG20230507123518.jpg
IMG20230507123521.jpg
IMG20230507123526.jpg
IMG2023050

In [ ]:
!zip -r /content/augmented.zip /content/images/new

  adding: content/images/new/ (stored 0%)
  adding: content/images/new/IMG_7263_4.xml (deflated 80%)
  adding: content/images/new/IMG20230504211234_4.xml (deflated 73%)
  adding: content/images/new/India_005829_3.xml (deflated 58%)
  adding: content/images/new/India_004074_2.jpg (deflated 5%)
  adding: content/images/new/India_007371_4.jpg (deflated 1%)
  adding: content/images/new/India_003999_3.xml (deflated 58%)
  adding: content/images/new/IMG_7462.xml (deflated 49%)
  adding: content/images/new/IMG20230504210955_2.xml (deflated 57%)
  adding: content/images/new/Japan_003777.xml (deflated 50%)
  adding: content/images/new/IMG20230504184636.xml (deflated 73%)
  adding: content/images/new/IMG_7256_4.xml (deflated 83%)
  adding: content/images/new/IMG_7389.xml (deflated 73%)
  adding: content/images/new/IMG_7284_4.xml (deflated 68%)
  adding: content/images/new/India_005811_4.xml (deflated 58%)
  adding: content/images/new/India_007066.jpg (deflated 5%)
  adding: content/images/new/In

In [ ]:
from google.colab import files

files.download('augmented.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp augmented.zip /content/gdrive/MyDrive/TP1